![Banner](images/banner.png)

# Working with Comma Separated Values (CSV) files

CSV is not a well-defined standard! 

"Unhelpful" (that's a joke) suggestions for Python programmers:
- Don't use CSV files: Keep the data in the database.
- Don't use Excel - use Oracle APEX
- Use Oracle Data Pump to load CSV files into Oracle Database
<hr>

Helpful suggestions:
- Python's ["csv" module](https://docs.python.org/3/library/csv.html) has extensive reading and writing support

In [ ]:
import cx_Oracle
import os
import platform
import time

if platform.system() == "Darwin":
    cx_Oracle.init_oracle_client(lib_dir=os.environ.get("HOME")+"/instantclient_19_8")
elif platform.system() == "Windows":
     cx_Oracle.init_oracle_client(lib_dir=r"C:\oracle\instantclient_19_14")

In [ ]:
un = "pythondemo"
pw = "welcome"
cs = "localhost/orclpdb1"

connection = cx_Oracle.connect(user=un, password=pw, dsn=cs)

## Reading CSV Files

In [ ]:
with connection.cursor() as cursor:
    try:
        cursor.execute("drop table t")
    except:
        ;

    cursor.execute("""create table t (k number, 
                                      first_name varchar2(30), 
                                      last_name varchar2(30), 
                                      country varchar2(30))""")

I have data in CSV file that looks like:
```
1,Fred,Nurke,UK
2,Henry,Crun,UK
```

The Python csv module has extensive functionality.  One sample is shown below.  For cx_Oracle users the important points are to use `executemany()` and send batches of rows to the database.  Tuning in your environment will determine the best batch size.

In [ ]:
import csv

# Adjust the batch size to meet your memory and performance requirements
batch_size = 10000

with connection.cursor() as cursor:
    
    sql = "insert into t (k, first_name, last_name, country) values (:1, :2, :3, :4)"
    
    # Predefine memory areas to match the table definition (or max data) to avoid reallocs
    cursor.setinputsizes(None, 30, 30, 30)

    with open("csv/data1.csv", "r") as csv_file:
        csv_reader = csv.reader(csv_file, delimiter=',')
        data = []
        for line in csv_reader:
            data.append((line[0], line[1], line[2], line[3]))   # e.g [('1', 'Fred', 'Nurke', 'UK')]
            if len(data) % batch_size == 0:
                cursor.executemany(sql, data)
                data = []
        if data:
            cursor.executemany(sql, data)
        connection.commit()

print("Done")

In [ ]:
# Check the results

with connection.cursor() as cursor:
    sql = "select * from t order by k"
    for r in cursor.execute(sql):
        print(r)

Tuning database features may also be beneficial. For example, disabling logging and/or indexes.

## Writing CSV Files

This just one way to write CSV files.  The important point for cx_Oracle users is to tune `cursor.arraysize` for your data and network.

In [ ]:
import time

sql = "select * from all_objects where rownum <= 10000"

with connection.cursor() as cursor:

    start=time.time()

    cursor.arraysize = 1

    with open("testwrite.csv", "w", encoding="utf-8") as outputfile:
        writer = csv.writer(outputfile, lineterminator="\n")
        results = cursor.execute(sql)
        writer.writerows(results)

    elapsed = time.time() - start
    print("Writing CSV: 10000 rows in {:06.4f} seconds".format(elapsed))  

If you change the array size and rerun the cell, the time taken may vary.

In [ ]:
# Confirm the number of lines in the output file is correct

r = os.system("wc -l testwrite.csv")